In [79]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import pandas as pd
import threading
from selenium.webdriver.chrome.options import Options

In [127]:
def get_sp500_tickers():
    # 读取维基百科上的标普500成分股列表
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    sp500_list = pd.read_html(url)[0]
    tickers = sp500_list['Symbol'].tolist()
    return tickers

sp500_tickers = get_sp500_tickers()

In [75]:
"""
dic = {}
for i in range(len(sp500_tickers)):
    print(i/len(sp500_tickers))
    if i%50==0:
        time.sleep(20)
    stock_code = sp500_tickers[i]
    url = "https://finance.yahoo.com/quote/"+stock_code+"/news"
    # 设置Selenium WebDriver
    driver = webdriver.Chrome()
    driver.get(url)

    time.sleep(3)  # 初始加载等待时间
    scroll_pause_time = 1  # 每次滚动后的等待时间
    max_scroll_attempts = 30  # 最大滚动次数，可以根据需要调整

    for _ in range(max_scroll_attempts):
        # 滚动一半屏幕高度
        driver.execute_script("window.scrollBy(0, document.body.scrollHeight / 2);")
        time.sleep(scroll_pause_time)

    # 获取完整的页面HTML
    html = driver.page_source
    driver.quit()

    # 使用BeautifulSoup解析HTML
    soup = BeautifulSoup(html, 'html.parser')

    # 提取新闻标题
    news_titles = []
    for tag in soup.find_all("a", class_="js-content-viewer"):
        if "mega-item-header-link" in tag.get("class", []) and "Fw(b)" in tag.get("class", []) and "Fz(18px)" in tag.get("class", []):
            title = tag.get_text(strip=True)
            if title:
                news_titles.append(title)
    dic[stock_code] = news_titles
"""

0.0


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.124)
Stacktrace:
	GetHandleVerifier [0x00007FF78D0C82B2+55298]
	(No symbol) [0x00007FF78D035E02]
	(No symbol) [0x00007FF78CEF05AB]
	(No symbol) [0x00007FF78CED0038]
	(No symbol) [0x00007FF78CF56BC7]
	(No symbol) [0x00007FF78CF6A15F]
	(No symbol) [0x00007FF78CF51E83]
	(No symbol) [0x00007FF78CF2670A]
	(No symbol) [0x00007FF78CF27964]
	GetHandleVerifier [0x00007FF78D440AAB+3694587]
	GetHandleVerifier [0x00007FF78D49728E+4048862]
	GetHandleVerifier [0x00007FF78D48F173+4015811]
	GetHandleVerifier [0x00007FF78D1647D6+695590]
	(No symbol) [0x00007FF78D040CE8]
	(No symbol) [0x00007FF78D03CF34]
	(No symbol) [0x00007FF78D03D062]
	(No symbol) [0x00007FF78D02D3A3]
	BaseThreadInitThunk [0x00007FF92F58257D+29]
	RtlUserThreadStart [0x00007FF9302CAA78+40]


In [128]:
dic = {}
with open('./data.json', 'r') as f:
    dic = json.load(f)

In [129]:
remain = []
for item in sp500_tickers:
    if item not in dic.keys():
        remain.append(item)
    elif dic[item]==[]:
        remain.append(item)

In [131]:
remain

['BRK.B', 'BF.B', 'WTW']

In [132]:
sp500_tickers = remain

In [133]:

# 全局字典用于存储结果
news_dict = {}

def fetch_news(stock_code):
    try:
        url = "https://finance.yahoo.com/quote/" + stock_code + "/news"
        driver = webdriver.Chrome()
        driver.get(url)

        time.sleep(3)  # 初始加载等待时间
        scroll_pause_time = 1  # 每次滚动后的等待时间
        max_scroll_attempts = 30  # 最大滚动次数

        for _ in range(max_scroll_attempts):
            driver.execute_script("window.scrollBy(0, document.body.scrollHeight / 2);")
            time.sleep(scroll_pause_time)

        html = driver.page_source
        driver.quit()

        soup = BeautifulSoup(html, 'html.parser')
        news_titles = []
        for tag in soup.find_all("a", class_="js-content-viewer"):
            if "mega-item-header-link" in tag.get("class", []) and "Fw(b)" in tag.get("class", []) and "Fz(18px)" in tag.get("class", []):
                title = tag.get_text(strip=True)
                if title:
                    news_titles.append(title)
        news_dict[stock_code] = news_titles
    except Exception as e:
        print(f"Error fetching data for {stock_code}: {e}")

def start_thread(ticker):
    thread = threading.Thread(target=fetch_news, args=(ticker,))
    thread.start()
    return thread

# 创建并启动线程，同时保持线程数量不超过20
max_threads = 10
active_threads = []

for i in range(len(sp500_tickers)):
    ticker = sp500_tickers[i]
    if len(active_threads) >= max_threads:
        # 等待任一线程结束
        active_threads[0].join()
        active_threads.pop(0)

    new_thread = start_thread(ticker)
    active_threads.append(new_thread)

    # 每50个股票后休息20秒
    if (i % 50)==0 and i!=0:
        time.sleep(20)

# 等待所有线程完成
for thread in active_threads:
    thread.join()


In [134]:
for item in dic.keys():
    if item not in news_dict.keys():
        news_dict[item] = dic[item]

In [135]:
len(news_dict)

503

In [136]:
rm = []
for item in news_dict.keys():
    if news_dict[item]==[]:
        rm.append(item)
rm

['BF.B', 'BRK.B']

In [137]:
# 写入 JSON 到文件
with open('./data.json', 'w') as f:
    json.dump(news_dict, f)